# Developing an Information Retrieval System with Document Ranking

This project aims to augment the Information Retrieval (IR) system developed in the previous assignments by incorporating different Document Ranking strategies. You should use the Cranfield collection as the dataset. You can find
that in the original format here or in the TREC XML format with binary tagging here.



## Project Overview

In this project, you will implement three different approaches for document ranking, including the vector space model,
the binary independence model, and the language model. Then, you need to compare these ranking models resorting
to the evaluation criteria in Lecture 7. Key components and functionalities are as follows:


### ⬜ Document Ranking – Language Model

You will implement a function for document ranking utilizing
the language model. The function will take as input a query text and an integer indicating the number of top
documents to be retrieved. You can choose between Dirichlet smoothing or Jelinek-Mercer smoothing to avoid
zeroes. You do not need to fine-tune parameters λ or α. Albeit, you need to discuss why your chosen methods
and parameters are preferred.



In [2]:
from pathlib import Path  # For standard paths that work on both windows and linux
import pickle  # For write/read dicts to/from files
import numpy as np
from nltk.tokenize import RegexpTokenizer  # For query tokenization
from nltk.stem import PorterStemmer  # For query stemming
import math # For calculating idf (log(N/df))
from collections import Counter # For calculating query's terms frequency

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
ps = PorterStemmer()

In [4]:
# reading posting list from file
# posting list has been created in the last step (preprocessing)
posting_list = None
docs = None
tokenized_docs = None

with open(Path("files") / "posting_list.pkl", "rb") as f:
        posting_list = pickle.load(f)

with open(Path("files") / "docs.pkl", "rb") as f:
        docs = pickle.load(f)

with open(Path("files") / "tokenized_docs.pkl", "rb") as f:
        tokenized_docs = pickle.load(f)

In [5]:
print(f"but({len(posting_list['but'])} docs): [doc_id: frequency]",posting_list["but"])

but(219 docs): [doc_id: frequency] {13: 2, 24: 1, 27: 1, 43: 1, 48: 2, 61: 1, 65: 1, 71: 2, 93: 1, 109: 2, 113: 1, 115: 1, 124: 1, 125: 1, 131: 1, 137: 1, 139: 1, 148: 1, 151: 1, 152: 1, 155: 2, 159: 1, 167: 1, 172: 1, 175: 1, 178: 1, 184: 1, 187: 1, 188: 1, 190: 1, 192: 1, 198: 2, 200: 1, 201: 4, 203: 1, 205: 1, 208: 1, 209: 1, 211: 2, 213: 1, 220: 1, 226: 1, 228: 1, 240: 1, 243: 1, 246: 1, 251: 2, 254: 2, 260: 1, 265: 1, 271: 1, 282: 1, 283: 1, 291: 1, 295: 1, 328: 4, 337: 2, 346: 1, 347: 1, 351: 1, 369: 1, 374: 1, 389: 1, 403: 2, 416: 1, 430: 1, 440: 1, 451: 1, 458: 2, 476: 2, 483: 1, 489: 1, 498: 2, 510: 1, 514: 1, 518: 1, 519: 1, 520: 1, 521: 1, 526: 1, 535: 1, 544: 1, 546: 1, 555: 1, 561: 1, 563: 1, 565: 1, 566: 1, 568: 1, 569: 1, 571: 1, 588: 1, 594: 1, 599: 1, 603: 1, 635: 1, 639: 1, 642: 1, 651: 1, 659: 1, 660: 1, 662: 1, 666: 1, 670: 1, 672: 1, 678: 1, 684: 2, 685: 1, 691: 1, 703: 2, 717: 1, 720: 1, 726: 1, 730: 1, 738: 1, 747: 1, 751: 1, 752: 2, 755: 1, 756: 2, 759: 1, 762: 

In [22]:
# List of all tokens in the collection. (Make it by combining document tokens)
collection_tokens = [j for sub in tokenized_docs for j in sub]
print(collection_tokens[:5], "+ ...")

['experiment', 'investig', 'of', 'the', 'aerodynam'] + ...


In [27]:
# Number of occurrences in the collection for each term.
collection_frequency = Counter(collection_tokens)
print({term: collection_frequency[term] for term in collection_tokens[:5]})

{'experiment': 470, 'investig': 524, 'of': 12671, 'the': 19444, 'aerodynam': 309}


In [28]:
# Probability of occurrence in the collection for each term.
p_term_collection = {
    key: collection_frequency[key] / len(collection_tokens)
    for key in collection_frequency
}
print({term:p_term_collection[term] for term in collection_tokens[:5]})

{'experiment': 0.0020734531818683137, 'investig': 0.002311679717657439, 'of': 0.055899415462666815, 'the': 0.0857791992941436, 'aerodynam': 0.001363185177015551}


In [49]:
# Sum of all probabilities must be one. (Just for testing)
sum([p_term_collection[term] for term in p_term_collection])

1.0000000000000562

In [45]:
# Probability of occurrence in the documents.
p_term_document = [
    {term: tf / len(doc_tokens) for term, tf in Counter(doc_tokens).items()}
    for doc_tokens in tokenized_docs
]

for i in range(3):
    print(f"doc_{i}:", p_term_document[i])

doc_0: {'experiment': 0.014388489208633094, 'investig': 0.007194244604316547, 'of': 0.07194244604316546, 'the': 0.08633093525179857, 'aerodynam': 0.007194244604316547, 'a': 0.050359712230215826, 'wing': 0.02158273381294964, 'in': 0.02877697841726619, 'slipstream': 0.03597122302158273, 'an': 0.02158273381294964, 'studi': 0.007194244604316547, 'propel': 0.007194244604316547, 'wa': 0.02877697841726619, 'made': 0.014388489208633094, 'order': 0.007194244604316547, 'to': 0.03597122302158273, 'determin': 0.007194244604316547, 'spanwis': 0.007194244604316547, 'distribut': 0.007194244604316547, 'lift': 0.02877697841726619, 'increas': 0.007194244604316547, 'due': 0.014388489208633094, 'at': 0.014388489208633094, 'differ': 0.02158273381294964, 'angl': 0.007194244604316547, 'attack': 0.007194244604316547, 'and': 0.007194244604316547, 'free': 0.007194244604316547, 'stream': 0.007194244604316547, 'veloc': 0.007194244604316547, 'ratio': 0.007194244604316547, 'result': 0.007194244604316547, 'were': 0.

In [50]:
# Sum of all probabilities must be one. (Just for testing)
for i in range(3):
    print(f"doc_{i}:", sum([p_term_document[i][term] for term in p_term_document[i]]))

doc_0: 0.9999999999999981
doc_1: 1.0000000000000009
doc_2: 1.0000000000000002


In [86]:
def lm_score(query, k, l):
    query = [ps.stem(token) for token in tokenizer.tokenize(query)]

    docs_scores = np.ones(len(docs))

    for doc_id in range(len(docs)):
        for term in query:
            docs_scores[doc_id] *= (
                l * (p_term_document[doc_id].get(term) or 0) + (1 - l) * p_term_collection[term]
            )

    result_doc_ids = docs_scores.argsort()[-k:][::-1]

    return [{"id": result_doc_ids[i], "score": docs_scores[result_doc_ids[i]]} for i in range(len(result_doc_ids))]

In [97]:
results = lm_score(
    "what is the basic mechanism of the transonic aileron buzz .",
    5,
    l = .5
)
results

[{'id': 495, 'score': 1.0944810384386183e-23},
 {'id': 902, 'score': 3.464452734490881e-26},
 {'id': 519, 'score': 9.723387370623949e-27},
 {'id': 312, 'score': 1.168978562256519e-27},
 {'id': 37, 'score': 9.542067045390059e-28}]

In [99]:
# Show the results
for i, result in enumerate(results):
    print(f"================== Result {i + 1} ==================")
    print(docs[result["id"]])
    print()

================== Result 1 ==================
a theory of transonic aileron buzz, neglecting viscous
effects .
  usaf-sponsored analysis of the unsteady perturbations of
two-dimensional transonic flow around an airfoil, where local supersonic
regions terminated by shock waves are present in the vicinity of the
airfoil .  viscous effects are neglected, and a linearized theory of
the perturbations due to harmonic oscillations of an aileron is
developed .  a series solution for the pressure distribution is obtained,
and numerical results for the nonsteady hinge moment, from the
first approximation to the solution, are presented .  as a result of
flutter analysis a stability boundary for transonic aileron buzz is
obtained .  comparison of the theoretical results with experimental
observations shows satisfactory agreement .

================== Result 2 ==================
two dimensional transonic unsteady flow with shock
waves .
  a study is made of the unsteady flow around an
airfoil at t